In [3]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import pandas as pd
import numpy as np
import plotly.io as pio
import plotly.offline.offline
import plotly.graph_objects as go

df = pd.read_csv("quadrant.csv",
                dtype = {'skills' : float,
                         'noc_name': str,
                         'brookfield_growth' : float,
                         'cluster' : str,
                         'noc_1' : int,
                         'degree' : float,
                         'emp_2016' : float,
                         'WFH': float})
df = df.sort_values(by=['cluster'])
df['cluster'] = df['cluster'].replace({'1': 'Service, Support and Communicators', '2': 'Classic Trades', '3': 'Research and STEM', '4': 'Business, Knowledge, and Operations', '5': 'Modern Trades'})
avincome = df['income'].median()

df['incomex']=df['income']-avincome


cluster_conditions = []
for i in df.cluster.unique():
    condition = df['cluster']== i
    cluster_conditions.append(condition)
cluster_conditions

pastelpalette = ['#ABC9E3','#FFA9A2','#B2E7A6','#E1C1EC','#FED9A6']
bgpalette = ['#093A55','#3680A9','#8CA5B3','#71B5DC','#CCE9F9']
#bwpalette = ['#000000','#4E4E4E','#989898','#C3C2C2','#DEDEDE']
df['colour'] = np.select(cluster_conditions, pastelpalette)

#bubble size and defining cluster data and names
bubble_size = []

for index, row in df.iterrows():
    bubble_size.append(row['emp_2016'])

df['size'] = bubble_size
cluster_names = df.cluster.unique()
cluster_data = {cluster:df.query("cluster == '%s'" %cluster)
                              for cluster in cluster_names}

sizeref = 2.*max(df['size'])/(90**2)

#skills and growth

fig = go.Figure()

for cluster_names, cluster in cluster_data.items():
    trace1=(go.Scatter(
            x=cluster['skills'],
            y=cluster['brookfield_growth'],
            meta = cluster['automation'],
            name=cluster_names,
            text=df['cluster'],
            customdata = cluster['noc_name'],
            hovertext = cluster['income'],
            hovertemplate=
            "<b>%{customdata}</b><br><br>" +
            "Skills Ratio: %{x:.2f}<br>" +
            "Growth Outlook: %{y:.2f}<br>" +
            "Employment (2016): %{marker.size:,}<br>" +
            "Income: %{hovertext:$,.2f}<br>"+
            "Automation: %{meta:.1%}"+
            "<extra></extra>",
            marker_size=cluster['size'],
            marker_color = cluster['colour']
            ))
    fig.add_trace(trace1)
    
fig.update_traces(
    mode='markers',
    marker={'sizemode':'area',
            'sizeref':sizeref})

fig.add_shape(
        # Line Horizontal
        dict(
            type="line",
            x0=df['skills'].min()*1.3,
            y0=0,
            x1=df['skills'].max()*1.1,
            y1=0,
            line=dict(
                color="black",
                width=1
            )))

fig.add_shape(
        # Line Vertical
        dict(
            type="line",
            x0=0,
            y0=df['brookfield_growth'].max()*1.3,
            x1=0,
            y1=df['brookfield_growth'].min()*1.3,
            line=dict(
                color="black",
                width=1
            )))

fig.update_layout(
    title='Occupational Clusters Quadrant',
    xaxis=dict(
        title='Skills Strength',
        gridwidth=2,
        gridcolor = 'white'
    ),
    yaxis=dict(
        title='',
        gridwidth=2,
        gridcolor = 'white'
    ),
    autosize=True,
    legend_title_text='Clusters:',
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=0,
        xanchor="right",
        x=1),
    margin=dict(l=70, r=30, t=30, b=30, pad=1),
    paper_bgcolor='rgb(243, 243, 243)',
    annotations=[
        dict(
            x=0,
            y=0.05,
            xref="paper",
            yref="y",
            text="Hands-on skills",
            font=dict(
                size=10,
                color="#0D2B53"),
            showarrow=False
        ),
        dict(
            x=1,
            y=0.05,
            xref="paper",
            yref="y",
            text="Science skills",
            font=dict(
                size=10,
                color="#0D2B53"),
            showarrow=False),
          dict(
            x=-0.06,
            y=0.5,
            xref="paper",
            yref="paper",
            text="Growth Outlook",
            textangle = -90,
            showarrow=False,
            font=dict(
                    size=14))
    ]
)